# COMP5318 Assignment 1: Classification

### Group number: A1part2 6  , SID1: 520080414   

In [1]:
import data as data
# Import all libraries
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from time import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import data
from pandas.core import frame
from sklearn.svm import SVC
from sklearn import datasets
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import numpy as np
import warnings
import pandas as pd
import nltk

In [2]:
# Configure libraries
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (10, 10)
plt.style.use('seaborn')

# Load dataset
df = pd.read_csv('training.300000.processed.noemoticon.csv')

# # Drop 'duration' column
# df = df.drop('date', axis=1).drop('query', axis=1)

print('Shape of dataframe:', df.shape)
df.head()

Shape of dataframe: (300000, 6)


,sentiment,id,date,query,username,text
0,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
1,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
2,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
3,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
4,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


In [3]:
# Pre-process dataset

df_tweet = df[['sentiment', 'text']]
sen_pos = df_tweet[df_tweet['sentiment'] == 4]
sen_neg = df_tweet[df_tweet['sentiment'] == 0]
print(len(sen_neg), len(sen_pos))

df_tweet = pd.concat([sen_pos, sen_neg])

start_time = time()
token = TweetTokenizer(reduce_len=True)

data = []

# Separating our features (text) and our labels into two lists to smoothen our work
X = df_tweet['text'].tolist()
Y = df_tweet['sentiment'].tolist()

# Building data list, that is a list of tuples, where each tuple is a pair of the tokenized text
# and its corresponding label
X = df_tweet['text'].tolist()
Y = df_tweet['sentiment'].tolist()

for x, y in zip(X,Y):
    if y == 4:
        data.append((token.tokenize(x), 1))
    else:
        data.append((token.tokenize(x), 0))

print('CPU Time:', time() - start_time)
data[:5]


149485 150515
CPU Time: 21.332491874694824


[(['on', 'lunch', '...', 'dj', 'should', 'come', 'eat', 'with', 'me'], 1),
 (['@mrstessyman',
   'thank',
   'you',
   'glad',
   'you',
   'like',
   'it',
   '!',
   'There',
   'is',
   'a',
   'product',
   'review',
   'bit',
   'on',
   'the',
   'site',
   'Enjoy',
   'knitting',
   'it',
   '!'],
  1),
 (['@PerezHilton',
   'Zach',
   'makes',
   'me',
   'pee',
   'sitting',
   'down',
   '!',
   'And',
   "I'm",
   'a',
   'grown',
   'gay',
   'man',
   '!'],
  1),
 (['to',
   'sum',
   'up',
   'my',
   'day',
   'in',
   'one',
   'word',
   '...',
   'kackered',
   '!'],
  1),
 (['@k9wkj', 'Great', 'minds', 'think', 'alike'], 1)]

In [4]:
# nltk.download('all')

In [5]:
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        # First, we will convert the pos_tag output tags to a tag format that the WordNetLemmatizer can interpret
        # In general, if a tag starts with NN, the word is a noun and if it stars with VB, the word is a verb.
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(data[0][0]))

['on', 'lunch', '...', 'dj', 'should', 'come', 'eat', 'with', 'me']


In [6]:
import re, string

# Stopwords are frequently-used words (such as “the”, “a”, “an”, “in”) that do not hold any meaning useful to extract sentiment.
# If it's your first time ever using nltk, you can download nltk's stopwords using: nltk.download('stopwords')
from nltk.corpus import stopwords
STOP_WORDS = stopwords.words('english')

# A custom function defined in order to fine-tune the cleaning of the input text. This function is highly dependent on each usecase.
# Note: Only include misspelling or abbreviations of commonly used words. Including many minimally present cases would negatively impact the performance.
def cleaned(token):
    if token == 'u':
        return 'you'
    if token == 'r':
        return 'are'
    if token == 'some1':
        return 'someone'
    if token == 'yrs':
        return 'years'
    if token == 'hrs':
        return 'hours'
    if token == 'mins':
        return 'minutes'
    if token == 'secs':
        return 'seconds'
    if token == 'pls' or token == 'plz':
        return 'please'
    if token == '2morow':
        return 'tomorrow'
    if token == '2day':
        return 'today'
    if token == '4got' or token == '4gotten':
        return 'forget'
    if token == 'amp' or token == 'quot' or token == 'lt' or token == 'gt' or token == '½25':
        return ''
    return token

In [7]:
# This function will be our all-in-one noise removal function
def remove_noise(tweet_token):
    cleaned_tokens = []

    for token, tag in pos_tag(tweet_token):
        # Eliminating the token if it is a link
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        # Eliminating the token if it is a mention
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        leammatizer = WordNetLemmatizer()
        token = leammatizer.lemmatize(token, pos)

        cleaned_token = cleaned(token.lower())

        # Eliminating the token if its length is less than 3, if it is a punctuation or if it is a stopword
        if cleaned_token not in string.punctuation and len(cleaned_token) > 2 and cleaned_token not in STOP_WORDS:
            cleaned_tokens.append(cleaned_token)

    return cleaned_tokens

# Prevewing the remove_noise() output
print(remove_noise(data[0][0]))

['lunch', '...', 'come', 'eat']


In [ ]:
start_time = time()
#unknowns
unks = []
UNKS = []

# As the Naive Bayesian classifier accepts inputs in a dict-like structure,
# we have to define a function that transforms our data into the required input structure
def list_to_dict(cleaned_tokens):
    return dict([token, True] for token in cleaned_tokens)

cleaned_tokens_list = []

# Removing noise from all the dat
for tokens, label in data:
    cleaned_tokens_list.append((remove_noise(tokens), label))

print('Removed Noise, CPU Time:', time() - start_time)
start_time = time()

final_data = []

# Transforming the data to fit the input structure of the Naive Bayesian classifier
for tokens, label in cleaned_tokens_list:
    final_data.append((list_to_dict(tokens),label))

print('Data Prepared for model, CPU Time:', time() - start_time)

# Previewing our final (tokenized, cleaned and lemmatized) data list
final_data[:5]


In [ ]:
def cleared(word):
    res = ''
    prev = None
    for char in word:
        if char == prev: continue
        prev = char
        res += char
    return res

def sentence_to_indices(sentence_words, word_to_index, max_len, i):
    global X, Y
    sentence_indices = []
    for j, w in enumerate(sentence_words):
        try:
            index = word_to_index[w]
        except:
            UNKS.append(w)
            w = cleared(w)
            try:
                index = word_to_index[w]
            except:
                index = word_to_index['unk']
                unks.append(w)
        X[i, j] = index

list_len = [len(i) for i, j in cleaned_tokens_list]
max_len = max(list_len)
X = np.zeros((len(cleaned_tokens_list), max_len))
Y = np.zeros((len(cleaned_tokens_list), ))

# for i, tk_lb in enumerate(cleaned_tokens_list):
#     tokens, label = tk_lb
#     # sentence_to_indices(tokens, word_to_index, max_len, i)
#     Y[i] = label

print(max_len)
print(X[:5])
print(Y[:5])
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify=Y)
# print(len(X_train))
# print(len(X_test))

In [8]:
import random

# As data is currently ordered by label, we have to shuffle it before splitting it
# .Random(140) randomizes our data with seed = 140. This guarantees the same shuffling for every execution of our code
random.Random(140).shuffle(final_data)

# 90% train data and 10% test data
trim_index = int(len(final_data) * 0.8)

train_data = final_data[:trim_index]
test_data = final_data[trim_index:]

In [15]:
# Training the model

start_time = time()

from nltk import classify
from nltk import NaiveBayesClassifier
from nltk import DecisionTreeClassifier
from nltk import SklearnClassifier
from nltk import MaxentClassifier
from sklearn.svm import LinearSVC

naive = NaiveBayesClassifier.train(train_data)
print('Naive Bayes Accuracy on train data:', classify.accuracy(naive, train_data))
print('Naive Bayes Accuracy on test data:', classify.accuracy(naive, test_data))

# sklearn = SklearnClassifier.train(train_data,)
# print('Naive Bayes Accuracy on train data:', sklearn.accuracy(naive, train_data))
# print('Naive Bayes Accuracy on test data:', sklearn.accuracy(naive, test_data))

decision = DecisionTreeClassifier.train(train_data)
print('Decision Tree Accuracy on train data:', classify.accuracy(decision, train_data))
print('DecisionT ree Accuracy on test data:', classify.accuracy(decision, test_data))

# maxent = MaxentClassifier.train(train_data)
# print('Maxent Accuracy on train data:', classify.accuracy(maxent, train_data))
# print('Maxent Accuracy on test data:', classify.accuracy(maxent, test_data))

print('\nCPU Time:', time() - start_time)

Naive Bayes Accuracy on train data: 0.8192583333333333
Naive Bayes Accuracy on test data: 0.7487333333333334


KeyboardInterrupt: 

In [ ]:
#Label Encoder
labels = train_data.target.unique().tolist()
labels.append("NEUTRAL")
labels

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

x_train = pad_sequences

encoder = LabelEncoder()
encoder.fit(train_data.target.tolist())

y_train = encoder.transform(train_data.target.tolist())
y_test = encoder.transform(test_data.target.tolist())

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [81]:
# Print first ten rows of pre-processed dataset to 4 decimal places as per assignment spec
# A function is provided to assist

# dataframe = df.replace('?', np.nan).replace('class1', 0).replace('class2', 1)
# simputer = SimpleImputer(missing_values=np.nan, strategy='mean')
# minmaxscaler = preprocessing.MinMaxScaler()
# dataset = imputer.fit_transform(dataframe)
# dataset = minmaxscaler.fit_transform(dataset)

# Select Features
feature = df_tweet.drop('sentiment', axis=1)

# Select Target
target = df_tweet['sentiment']

# Set Training and Testing Data
X_train, X_test, y_train, y_test = train_test_split(feature , target,
                                                    shuffle = True,
                                                    test_size=0.2,
                                                    random_state=1)

# Show the Training and Testing Data
print('Shape of training feature:', X_train.shape)
print('Shape of testing feature:', X_test.shape)
print('Shape of training label:', y_train.shape)
print('Shape of training label:', y_test.shape)

def print_data(X, n_rows=10):
    """Takes a numpy data array and target and prints the first ten rows.

    Arguments:
        X: numpy array of shape (n_examples, n_features)
        y: numpy array of shape (n_examples)
        n_rows: numpy of rows to print
    """
    for example_num in range(n_rows):
        for feature in X[example_num][0:-1:1]:
            print("{:.4f}".format(feature), end=",")
        print(int(X[example_num][-1]))


# print_data(dataset)


Shape of training feature: (240000, 5)
Shape of testing feature: (60000, 5)
Shape of training label: (240000,)
Shape of training label: (60000,)


### Part 1: Cross-validation without parameter tuning

In [28]:
## Setting the 10 fold stratified cross-validation
cvKFold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
# The stratified folds from cvKFold should be provided to the classifiers

In [29]:
# Logistic Regression
def logregClassifier(X, y):
    logreg = LogisticRegression(random_state=0)
    scores = cross_val_score(logreg, np.asarray(X, dtype='float64'), y, cv=cvKFold)
    return scores.mean()

In [30]:
#Naïve Bayes
def nbClassifier(X, y):
    nb = GaussianNB()
    scores = cross_val_score(nb, np.asarray(X, dtype='float64'), y, cv=cvKFold)
    return scores.mean()

In [31]:
# Decision Tree
def dtClassifier(X, y):
    classifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
    scores = cross_val_score(classifier, np.asarray(X, dtype='float64'), y, cv=cvKFold)
    return scores.mean()

In [32]:
# Ensembles: Bagging, Ada Boost and Gradient Boosting
def bagDTClassifier(X, y, n_estimators, max_samples, max_depth):
    classifier = BaggingClassifier(DecisionTreeClassifier(max_depth=max_depth, criterion='entropy', random_state=0)
                                   , n_estimators=n_estimators, max_samples=max_samples, random_state=0)
    scores = cross_val_score(classifier, np.asarray(X, dtype='float64'), y, cv=cvKFold)
    return scores.mean()


def adaDTClassifier(X, y, n_estimators, learning_rate, max_depth):
    classifier = AdaBoostClassifier(DecisionTreeClassifier(max_depth=max_depth, criterion='entropy', random_state=0)
                                    , n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
    scores = cross_val_score(classifier, np.asarray(X, dtype='float64'), y, cv=cvKFold)
    return scores.mean()


def gbClassifier(X, y, n_estimators, learning_rate):
    classifier = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
    scores = cross_val_score(classifier, np.asarray(X, dtype='float64'), y, cv=cvKFold)
    return scores.mean()

### Part 1 Results

In [87]:
from sklearn.preprocessing import LabelEncoder

# Parameters for Part 1:
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

#Bagging
bag_n_estimators = 60
bag_max_samples = 100
bag_max_depth = 6

#AdaBoost
ada_n_estimators = 60
ada_learning_rate = 0.5
ada_bag_max_depth = 6

#GB
gb_n_estimators = 60
gb_learning_rate = 0.5

# Print results for each classifier in part 1 to 4 decimal places here:
print("LogR average cross-validation accuracy: {:.4f}".format(logregClassifier(X_train, y_train)))
print("NB average cross-validation accuracy: {:.4f}".format(nbClassifier(X_train, y_train)))
print("DT average cross-validation accuracy: {:.4f}".format(dtClassifier(X_train, y_train)))
print("Bagging average cross-validation accuracy: {:.4f}".format(
    bagDTClassifier(X_train, y_train, bag_n_estimators, bag_max_samples, bag_max_depth)))
print("AdaBoost average cross-validation accuracy: {:.4f}".format(
    adaDTClassifier(X_train, y_train, ada_n_estimators, ada_learning_rate, ada_bag_max_depth)))
print("GB average cross-validation accuracy: {:.4f}".format(gbClassifier(X_train, y_train, gb_n_estimators, gb_learning_rate)))

ValueError: could not convert string to float: 'Sun May 31 04:56:09 PDT 2009'

### Part 2: Cross-validation with parameter tuning

In [33]:
# KNN
k = [1, 3, 5, 7, 9]
p = [1, 2]


def bestKNNClassifier(X, y):
    param_grid = {'n_neighbors': k, 'p': p}
    classifier = KNeighborsClassifier()
    grid_search= GridSearchCV(classifier, param_grid, cv=cvKFold, return_train_score=True)
    grid_search.fit(X, y)
    return grid_search

In [34]:
# SVM
# You should use SVC from sklearn.svm with kernel set to 'rbf'
C = [0.01, 0.1, 1, 5, 15]
gamma = [0.01, 0.1, 1, 10, 50]


def bestSVMClassifier(X, y):
    param_grid = {'C': C, 'gamma': gamma}
    grid_search = GridSearchCV(SVC(kernel='rbf', random_state=0), param_grid, cv=cvKFold)
    grid_search.fit(X, y)
    return grid_search

In [35]:
# Random Forest
# You should use RandomForestClassifier from sklearn.ensemble with information gain and max_features set to ‘sqrt’.
n_estimators = [10, 30, 60, 100, 150]
max_leaf_nodes = [6, 12, 18]


def bestRFClassifier(X, y):
    param_grid = {'n_estimators': n_estimators, 'max_leaf_nodes': max_leaf_nodes}
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
    grid_search = GridSearchCV(RandomForestClassifier(random_state=0, criterion='entropy'), param_grid, cv=cvKFold,
                               return_train_score=True)
    grid_search.fit(X_train, y_train)
    return grid_search

### Part 2: Results

In [36]:
# Perform Grid Search with 10-fold stratified cross-validation (GridSearchCV in sklearn).
# The stratified folds from cvKFold should be provided to GridSearchV

# This should include using train_test_split from sklearn.model_selection with stratification and random_state=0
# Print results for each classifier here. All results should be printed to 4 decimal places except for
# "k", "p", n_estimators" and "max_leaf_nodes" which should be printed as integers.
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

bestKNN = bestKNNClassifier(X_train, y_train)
print("KNN best k: ", bestKNN.best_params_['n_neighbors'])
print("KNN best p: ", bestKNN.best_params_['p'])
print("KNN cross-validation accuracy: {:.4f}".format(bestKNN.best_score_))
print("KNN test set accuracy: {:.4f}".format(bestKNN.score(X_test, y_test)))

print()
bestSVM = bestSVMClassifier(X_train, y_train)
print("SVM best C: {:.4f}".format(bestSVM.best_params_['C']))
print("SVM best gamma: {:.4f}".format(bestSVM.best_params_['gamma']))
print("SVM cross-validation accuracy: {:.4f}".format(bestSVM.best_score_))
print("SVM test set accuracy: {:.4f}".format(bestSVM.score(X_test, y_test)))

print()
bestRFC = bestRFClassifier(X_train, y_train)
y_predict = bestRFC.predict(X_test)
print("RF best n_estimators: ", bestRFC.best_params_['n_estimators'])
print("RF best max_leaf_nodes: ", bestRFC.best_params_['max_leaf_nodes'])
print("RF cross-validation accuracy: {:.4f}".format(bestRFC.best_score_))
print("RF test set accuracy: {:.4f}".format(bestRFC.score(X_test, y_test)))
print("RF test set macro average F1: {:.4f}".format(f1_score(y_test, y_predict, average='macro')))
print("RF test set weighted average F1: {:.4f}".format(f1_score(y_test, y_predict, average='weighted')))

KNN best k:  1
KNN best p:  1
KNN cross-validation accuracy: 1.0000
KNN test set accuracy: 1.0000



ValueError: 
All the 250 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
250 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\FU\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\FU\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py", line 201, in fit
    y = self._validate_targets(y)
  File "C:\Users\FU\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py